# World Bank's World Development Indicators vs. Environment Analysis - Analysis Notebook

In [137]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

In [138]:
%run ./help_funcs.ipynb

## Load the datasets

| Dataset | Series Code | Series Description/Name |
| --- | --- | --- |
|CO2| EN.ATM.CO2E.KT | CO2 emissions (kt)|
|Other Gas | EN.ATM.GHGT.KT.CE | Total greenhouse gas emissions (kt of CO2 equivalent) |
|Other Gas | EN.ATM.METH.KT.CE | Methane emissions (kt of CO2 equivalent)|
|Other Gas | EN.ATM.NOXE.KT.CE | Nitrous oxide emissions (thousand metric tons of CO2 equivalent)|

